In [ ]:
import os
import zipfile

In [ ]:
local_zip1= '/content/train.zip'

zip1= zipfile.ZipFile(local_zip1,'r')
zip1.extractall('./')
zip1.close()


In [ ]:
local_zip2= '/content/valid.zip'

zip1= zipfile.ZipFile(local_zip2,'r')
zip1.extractall()
zip1.close()


In [ ]:
local_zip3= '/content/test.zip'

zip3= zipfile.ZipFile(local_zip3,'r')
zip3.extractall()
zip3.close()

In [ ]:
"""
!rm -rf '/content/test'
!rm -rf '/content/valid'
!rm -rf '/content/train'
!rm -rf '/content/content/brain_tumor'
"""

"\n!rm -rf '/content/test'\n!rm -rf '/content/valid'\n!rm -rf '/content/train'\n!rm -rf '/content/content/brain_tumor'\n"

In [ ]:
base_dir= './'
train_dir=os.path.join(base_dir,'train')
validation_dir=os.path.join(base_dir,'valid')
test_dir= os.path.join(base_dir,'test')

In [ ]:
train_GT_dir=os.path.join(train_dir,'glioma_tumor')
train_MT_dir=os.path.join(train_dir,'meningioma_tumor')
train_NT_dir=os.path.join(train_dir,'no_tumor')
train_PT_dir=os.path.join(train_dir,'pituitary_tumor')


validation_GT_dir=os.path.join(validation_dir,'glioma_tumor')
validation_MT_dir=os.path.join(validation_dir,'meningioma_tumor')
validation_NT_dir=os.path.join(validation_dir,'no_tumor')
validation_PT_dir=os.path.join(validation_dir,'pituitary_tumor')

test_GT_dir=os.path.join(test_dir,'glioma_tumor')
test_MT_dir=os.path.join(test_dir,'meningioma_tumor')
test_NT_dir=os.path.join(test_dir,'no_tumor')
test_PT_dir=os.path.join(test_dir,'pituitary_tumor')

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
img_input= layers.Input(shape=(150,150,3))
##first Conv layer
L= layers.Conv2D(16,3,activation='relu')(img_input) ##img_inpu is previous layer ##16 is number of filetrs
L= layers.MaxPooling2D(2)(L)

##second Conv
L= layers.Conv2D(32,3,activation='relu')(L)
L= layers.MaxPooling2D(2)(L)


##final one with 64 filetrs with individual 3*3 filters kernals

L= layers.Conv2D(64,3,activation='relu')(L) ## Good feature with 'relu' they resist vanishing gradients
L= layers.MaxPooling2D(2)(L)

In [ ]:
F= layers.Flatten()(L)
F= layers.Dense(512,activation='relu')(F)
output=layers.Dense(4,activation='softmax')(F)

In [ ]:
model= Model(img_input,output)
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001),metrics=['acc'] )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen= ImageDataGenerator(rescale=1./255)
val_datagen= ImageDataGenerator(rescale=1./255)


train_generator= train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical'
)

validation_generator= val_datagen.flow_from_directory(
    validation_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical'
)

In [ ]:
history= model.fit_generator(
    train_generator,
    steps_per_epoch= train_generator.samples/train_generator.batch_size,
    epochs=15,
    validation_data= validation_generator,
    validation_steps=validation_generator.samples/validation_generator.batch_size, verbose=2
)

In [ ]:
test_datagen= ImageDataGenerator(rescale=1./255)

test_generator= test_datagen.flow_from_directory(
    test_dir,
    target_size=(150,150),
    batch_size=20,
    class_mode='categorical'
)

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print('Test accuracy :', accuracy)

In [ ]:
print(test_generator)

In [ ]:
pred = model.predict(test_generator)

In [ ]:
import numpy as np
y_class=[]
for i in pred:
  y_class.append(np.argmax(i))
print(len(y_class))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


def confusion_matrix(test_y, y_class):
  plt.figure(figsize=(10,7))
  y_actu = pd.Series(test_y, name='Actual')
  y_pred = pd.Series(y_class, name='Predicted')
  cm = pd.crosstab(y_actu, y_pred)
  ax = sns.heatmap(cm, annot=True, fmt="d")
  plt.ylabel('True label', fontsize=18, fontweight="bold")
  plt.xlabel('Predicted label',fontsize=18, fontweight="bold")
  #plt.savefig("/content/drive/MyDrive/cm")


y_true = test_generator.classes
y_pred = y_class
confusion_matrix(y_pred, y_true)